### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [13]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [14]:
X_test[['humidity', 'sunshine', 'pressure']].head(3)

,humidity,sunshine,pressure
0,39.0,13.2,1014.0
1,8.0,10.8,1013.7
2,62.0,10.1,1009.6


In [15]:
import urllib.request
import json      

def get_prediction(humidity, sunshine, pressure):
    body = {'humidity': humidity, 
                            'sunshine': sunshine,
                            'pressure': pressure} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [21]:
get_prediction(62.0, 1.2, 1009.0)

0.34

In [17]:
%%time
predictions = X_test[['humidity', 'sunshine', 'pressure']].iloc[:50].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

CPU times: user 39.8 ms, sys: 581 µs, total: 40.4 ms
Wall time: 1.88 s


In [19]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:50])

0.8253638253638254

Давайте подумаем, где наше api может сломаться

In [20]:
get_prediction("", "", None)

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR